In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle

from fastkaggle import *

In [3]:
from fastai.imports import *
np.set_printoptions(linewidth=130)
import os
from pathlib import Path
path = Path('/kaggle/input/paddy-doctor/paddy-disease-classification')
path

Path('/kaggle/input/paddy-doctor/paddy-disease-classification')

In [4]:
from fastai.vision.all import *
set_seed(42)

path.ls()

(#6) [Path('/kaggle/input/paddy-doctor/paddy-disease-classification/sample_submission.csv'),Path('/kaggle/input/paddy-doctor/paddy-disease-classification/train_images'),Path('/kaggle/input/paddy-doctor/paddy-disease-classification/.jovianrc'),Path('/kaggle/input/paddy-doctor/paddy-disease-classification/.ipynb_checkpoints'),Path('/kaggle/input/paddy-doctor/paddy-disease-classification/train.csv'),Path('/kaggle/input/paddy-doctor/paddy-disease-classification/test_images')]

In [5]:
tst_files = get_image_files(path/'test_images').sorted()

In [6]:
df = pd.read_csv(path/'train.csv')
df.label.value_counts()

label
normal                      1764
blast                       1738
hispa                       1594
dead_heart                  1442
tungro                      1088
brown_spot                   965
downy_mildew                 620
bacterial_leaf_blight        479
bacterial_leaf_streak        380
bacterial_panicle_blight     337
Name: count, dtype: int64

In [7]:
trn_path = path/'train_images'/'bacterial_panicle_blight'

In [8]:
def train(arch, size, item=Resize(480, method='squish'), accum=1, finetune=True, epochs=12):
    dls = ImageDataLoaders.from_folder(trn_path, valid_pct=0.2, item_tfms=item,
        batch_tfms=aug_transforms(size=size, min_scale=0.75), bs=64//accum)
    cbs = GradientAccumulation(64) if accum else []
    learn = vision_learner(dls, arch, metrics=error_rate, cbs=cbs).to_fp16()
    if finetune:
        learn.fine_tune(epochs, 0.01)
        return learn.tta(dl=dls.test_dl(tst_files))
    else:
        learn.unfreeze()
        learn.fit_one_cycle(epochs, 0.01)

In [9]:
train('convnext_small_in22k', 128, epochs=1, accum=1, finetune=False)

/usr/local/lib/python3.11/dist-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name convnext_small_in22k to current convnext_small.fb_in22k.
  model = create_fn(
/usr/local/lib/python3.11/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.11/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:03


In [10]:
import gc
def report_gpu():
    print(torch.cuda.list_gpu_processes())
    gc.collect()
    torch.cuda.empty_cache()

In [11]:
report_gpu()

GPU:0
process       5941 uses     3248.000 MB GPU memory


In [12]:
train('convnext_small_in22k', 128, epochs=1, accum=2, finetune=False)
report_gpu()

epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:02


GPU:0
process       5941 uses     2188.000 MB GPU memory


In [13]:
train('convnext_small_in22k', 128, epochs=1, accum=4, finetune=False)
report_gpu()

epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:02


GPU:0
process       5941 uses     1664.000 MB GPU memory


In [14]:
train('convnext_large_in22k', 224, epochs=1, accum=2, finetune=False)
report_gpu()

/usr/local/lib/python3.11/dist-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name convnext_large_in22k to current convnext_large.fb_in22k.
  model = create_fn(


epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:06


GPU:0
process       5941 uses    10072.000 MB GPU memory


In [15]:
train('convnext_large_in22k', (320,240), epochs=1, accum=2, finetune=False)
report_gpu()

epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:09


GPU:0
process       5941 uses    13396.000 MB GPU memory


In [16]:
train('vit_large_patch16_224', 224, epochs=1, accum=2, finetune=False)
report_gpu()

epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:08


GPU:0
process       5941 uses    11500.000 MB GPU memory


In [17]:
train('swinv2_large_window12_192_22k', 192, epochs=1, accum=2, finetune=False)
report_gpu()

/usr/local/lib/python3.11/dist-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name swinv2_large_window12_192_22k to current swinv2_large_window12_192.ms_in22k.
  model = create_fn(


epoch,train_loss,valid_loss,error_rate,time


RuntimeError: running_mean should contain 12 elements not 3072

In [18]:
train('swin_large_patch4_window7_224', 224, epochs=1, accum=2, finetune=False)
report_gpu()

model.safetensors:   0%|          | 0.00/788M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,error_rate,time


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 12.12 MiB is free. Process 5941 has 14.73 GiB memory in use. Of the allocated memory 14.23 GiB is allocated by PyTorch, and 326.43 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [19]:
res = 640,480

In [20]:
models = {
    'convnext_large_in22k': {
        (Resize(res), (320,224)),
    }, 'vit_large_patch16_224': {
        (Resize(480, method='squish'), 224),
        (Resize(res), 224),
    }, 'swinv2_large_window12_192_22k': {
        (Resize(480, method='squish'), 192),
        (Resize(res), 192),
    }, 'swin_large_patch4_window7_224': {
        (Resize(res), 224),
    }
}

In [21]:
trn_path = path/'train_images'

In [22]:
tta_res = []

for arch,details in models.items():
    for item,size in details:
        print('---',arch)
        print(size)
        print(item.name)
        tta_res.append(train(arch, size, item=item, accum=2)) #, epochs=1))
        gc.collect()
        torch.cuda.empty_cache()

--- convnext_large_in22k
(320, 224)
Resize -- {'size': (480, 640), 'method': 'crop', 'pad_mode': 'reflection', 'resamples': (<Resampling.BILINEAR: 2>, <Resampling.NEAREST: 0>), 'p': 1.0}


OutOfMemoryError: Exception occured in `TrainEvalCallback` when calling event `before_fit`:
	CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 6.12 MiB is free. Process 5941 has 14.73 GiB memory in use. Of the allocated memory 14.28 GiB is allocated by PyTorch, and 283.62 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)